## Fake NEws Classificer USsing Bidirectional LSTM

In [22]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [23]:
df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)

<ipython-input-23-772ef6e293b2>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)
Skipping line 7958: unexpected end of data


In [24]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [25]:
df.isnull().sum()

id          0
title     216
author    753
text       14
label       0
dtype: int64

In [26]:
df.shape

(7956, 5)

In [27]:
df=df.dropna()

In [28]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [29]:
## get the independent features
X=df.drop('label',axis=1)

In [30]:
y=df['label']

In [31]:
## Check whether dataset is balanced or not
y.value_counts()

0    3980
1    3007
Name: label, dtype: int64

In [32]:
import tensorflow as tf

In [33]:
tf.__version__

'2.12.0'

In [34]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [35]:
## vocabulary size
voc_size=5000

In [36]:
messages=X.copy()

In [37]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [38]:
import nltk
import re
from nltk.corpus import stopwords

In [39]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
messages.reset_index(inplace=True)

In [41]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176

In [42]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [43]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4161, 4387, 4951, 4414, 1013, 2644, 1467, 4996, 1748, 1844],
 [2966, 4546, 3600, 4463, 42, 184, 3408],
 [3215, 1831, 1596, 3981],
 [112, 2845, 1861, 4402, 856, 3907],
 [4841, 42, 2628, 1440, 784, 2310, 42, 1, 3574, 561],
 [1671,
  3918,
  726,
  2086,
  3785,
  2797,
  840,
  4506,
  2697,
  3067,
  1155,
  1684,
  2100,
  733,
  3408],
 [2002, 2587, 474, 1448, 3256, 218, 4750, 3845, 1119, 2808, 1076],
 [2914, 346, 3096, 452, 4680, 1534, 2797, 548, 1119, 2808, 1076],
 [3853, 3055, 2932, 2840, 2290, 2353, 790, 80, 2797, 3624],
 [4980, 3517, 564, 1436, 2588, 1306, 2306, 4517],
 [52, 3843, 2329, 3583, 436, 2903, 1657, 3469, 3262, 3557, 2814],
 [4402, 327, 1013, 2353, 2797, 4680],
 [308, 3914, 1031, 2130, 1285, 3848, 2405, 3062, 3154],
 [4323, 899, 1234, 3315, 4720, 627, 4783, 1119, 2808, 1076],
 [3237, 2418, 3796, 2471, 1035, 1119, 2808, 1076],
 [3583, 1444, 2768, 4988, 3034, 4246, 3749, 4483, 3225, 2147],
 [2928, 1414, 4546],
 [1758, 3524, 3641, 1450, 2797, 3298, 4397, 3408],
 [2299, 3

## Embedding Representation

In [44]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 4996, 1748, 1844],
       [   0,    0,    0, ...,   42,  184, 3408],
       [   0,    0,    0, ..., 1831, 1596, 3981],
       ...,
       [   0,    0,    0, ..., 1119, 2808, 1076],
       [   0,    0,    0, ..., 2844, 3600, 2476],
       [   0,    0,    0, ..., 4689,  172,  171]], dtype=int32)

In [45]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [47]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [48]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
147/147 [==============================] - 22s 85ms/step - loss: 0.3817 - accuracy: 0.8150 - val_loss: 0.2197 - val_accuracy: 0.9037
Epoch 2/10
147/147 [==============================] - 3s 19ms/step - loss: 0.1685 - accuracy: 0.9323 - val_loss: 0.2184 - val_accuracy: 0.9128
Epoch 3/10
147/147 [==============================] - 3s 22ms/step - loss: 0.0809 - accuracy: 0.9737 - val_loss: 0.2699 - val_accuracy: 0.9133
Epoch 4/10
147/147 [==============================] - 2s 14ms/step - loss: 0.0380 - accuracy: 0.9893 - val_loss: 0.3525 - val_accuracy: 0.9024
Epoch 5/10
147/147 [==============================] - 2s 14ms/step - loss: 0.0229 - accuracy: 0.9934 - val_loss: 0.3885 - val_accuracy: 0.9068
Epoch 6/10
147/147 [==============================] - 2s 15ms/step - loss: 0.0176 - accuracy: 0.9947 - val_loss: 0.4660 - val_accuracy: 0.9068
Epoch 7/10
147/147 [==============================] - 2s 11ms/step - loss: 0.0147 - accuracy: 0.9964 - val_loss: 0.5188 - val_accuracy: 0.890

In [49]:
y_pred=model.predict(X_test)

73/73 [==============================] - 1s 3ms/step


In [50]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [51]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1183,  129],
       [  96,  898]])

In [52]:
print(accuracy_score(y_test,y_pred))

0.9024284475281873
